In [1]:
import numpy as np
import scipy, scipy.io
import mne
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
from scipy import fft, ifft, arange
from statsmodels.tsa.api import VAR, DynamicVAR

signal1 = mne.read_epochs('/home/robertofelipe_sg/Documents/Python/P04_Bsl_preprocessed_epo.fif')
                          #P22_TACS_preprocessed_epo.fif')

# Get data as a numpy array --> trials x channels x time-points
data = signal1['stim_cor'].get_data()
print(data.shape)

# Retrieve channels of interest 
chan2use = ['PO4', 'CP4']
channels = signal1.info['ch_names']
ch_index = np.empty((len(chan2use),1), dtype=int)
i = 0
for idx, chan in enumerate(channels):
    if chan in chan2use:
        ch_index[i,0] = int(idx)
        i=i+1

# Frequency Parameters
min_freq = 2
max_freq = 42
num_freq = 40
frex = np.linspace(min_freq, max_freq, num_freq)

# Baseline
baseline_window = np.array([-0.3, -0.1]);

# Baseline time into indices
baseidx1 = int(((baseline_window[0]+1.5)*data.shape[2])/3)
baseidx2 = int(((baseline_window[1]+1.5)*data.shape[2])/3)

# Parameters Morlet Wavelet
cyc_rng = np.array([3, 10])
srate = signal1.info['sfreq']
timew = arange(-2,2,1/srate)
s = (np.logspace(np.log10(cyc_rng[0]), np.log10(cyc_rng[1]), num_freq)) / (2*np.pi*frex)
half_w = (len(timew)/2) +1
half_w = int(half_w)

# Initialize output Time-Frequency
tifr = np.empty((len(frex), data.shape[2]))
itpc = np.empty((len(frex), data.shape[2]))
ispc = np.empty((len(frex), data.shape[2]))
phase_data = np.empty((len(ch_index),len(frex), data.shape[0], data.shape[2]), dtype=complex)
tifrx = np.empty((len(ch_index), len(frex), data.shape[2]), dtype=int)
mean_tf = np.empty((len(frex), data.shape[2]))
nData = data.shape[0] * data.shape[2] #* len(ch_index)
nConv = len(timew) + nData - 1
areg = np.empty((data.shape[0], len(ch_index), data.shape[2]))

# Looping over channels
for ch in range(ch_index.shape[0]):

    # FFT on all trials concatenated
    alldata = np.reshape(data[:,ch_index[ch,0],:], (1,nData))
    areg[:,ch,:] = data[:,ch_index[ch,0],:]
    datax = fft(alldata, nConv)
    
    # Looping over Frequencies
    for idx, f in enumerate(frex):

        # Create best wavelet according to frequency of analysis (best resolution)
        wavelet = (np.exp(2j*np.pi*f*timew)) * (np.exp((-timew**2)/(2*s[idx]**2)))
        wavex = fft(wavelet, nConv)
        wavex = wavex / max(wavex)

        # Applying convolution in the Freq.Domain
        convo = ifft(datax*wavex)
        convo = convo[0, half_w-1 : len(convo)-half_w+1]

        # Reshape into trials x time
        convo = np.reshape(convo, (data.shape[0], data.shape[2]))

        # Average over all trials
        tifr[idx,:] = np.mean(np.abs(convo)**2, axis=0)
        
        # Compute ITPC -> Mean over trials
        itpc[idx,:] = abs(np.mean(np.exp(1j*(np.angle(convo))),0))
               
        # Calculate angle for every channel
        phase_data[ch,idx,:,:] = np.angle(convo)
            
    # Decibel Normalization    
    tifrx[ch,:,:] = (10*np.log10([tifr[:,i] / (np.mean(tifr[:,baseidx1:baseidx2],1)) for i in range(data.shape[2])])).T

    # Average over channels
    mean_tf[:,:] = np.mean(tifrx, axis=0)
            
    # Plotting Spectrum
    fig1 = plt.figure()
    ax = fig1.add_subplot(3,1,ch+1)
    CS = plt.contourf(signal1.times[50:700], frex, itpc[:,50:700], 40)
    cbar = fig1.colorbar(CS)
    ax.set_title('%s' %chan2use[ch])
    
# Phase Difference
ph_diff = phase_data[1,:,:,:] - phase_data[0,:,:,:]
ispc_freq = abs((np.mean(np.exp(1j*ph_diff),1)))# frex x trials x time-points
ispc_alpha = np.angle(np.mean(np.mean(np.exp(1j*ph_diff),1)[5:11],0))
ispc_gamma = np.angle(np.mean(np.mean(np.exp(1j*ph_diff),1)[28:40],0))
ispc_time = abs(np.mean(np.mean(np.exp(1j*ph_diff),1),0))
ispc_tri = abs(np.mean(np.mean(np.exp(1j*ph_diff[:,:,372:378]),2),0))
ispc_ang_tr = np.angle(np.mean(np.mean(np.exp(1j*ph_diff[:,:,372:378]),2),0))

#scipy.io.savemat('areg.mat', {'electrodes':areg})

# fig2 = plt.figure()
# ax1 = fig2.add_subplot(1,2,1)
# stdev = np.std(ispc_time)
# plt.plot(signal1.times[50:700], ispc_time[50:700])
# plt.fill_between(signal1.times[50:700], ispc_time[50:700]+stdev, ispc_time[50:700]-stdev,alpha=.1)
# #ax1.plot(frex, ispc_freq)
# plt.tick_params(labelsize=20)
# plt.ylabel('ISPC', fontsize=20)
# plt.xlabel('Time(s)', fontsize=20)
# plt.ylim(0,0.5)
# ax2 = fig2.add_subplot(1,2,2, projection='polar')

# phase=np.empty([2,data.shape[0]])
# magni=np.empty([2,data.shape[0]])
# for c,_ in enumerate(ispc_tri):
#     phase[:,c]=np.array([0, ispc_ang_tr[c]])
#     magni[:,c]=np.array([0, ispc_tri[c]])
# colors = magni
# plt.scatter(phase,magni,c=colors, marker='o')

# fig3 = plt.figure()
# plt.plot(signal1.times[50:700], ispc_alpha[50:700])
# plt.plot(signal1.times[50:700], ispc_gamma[50:700])
# plt.ylim(2*np.pi,-2*np.pi)

# plt.show()

Reading /home/robertofelipe_sg/Documents/Python/P04_Bsl_preprocessed_epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1496.00 ms
        0 CTF compensation matrices available
147 matching events found
Applying baseline correction (mode: mean)
147 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
(49, 64, 750)


In [2]:
channels

['FP1',
 'FP2',
 'F3',
 'F4',
 'C3',
 'C4',
 'P3',
 'P4',
 'O1',
 'O2',
 'F7',
 'F8',
 'T7',
 'T8',
 'P7',
 'P8',
 'FZ',
 'CZ',
 'PZ',
 'OZ',
 'FC1',
 'FC2',
 'CP1',
 'CP2',
 'FC5',
 'FC6',
 'CP5',
 'CP6',
 'TP9',
 'TP10',
 'POZ',
 'ECG',
 'F1',
 'F2',
 'C1',
 'C2',
 'P1',
 'P2',
 'AF3',
 'AF4',
 'FC3',
 'FC4',
 'CP3',
 'CP4',
 'PO3',
 'PO4',
 'F5',
 'F6',
 'C5',
 'C6',
 'P5',
 'P6',
 'AF7',
 'AF8',
 'FT7',
 'FT8',
 'TP7',
 'TP8',
 'PO7',
 'PO8',
 'FT9',
 'FT10',
 'FPZ',
 'FCZ']

In [ ]:
import numpy as np
import scipy as sp
import mne
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import fft, ifft, arange, stats, signal
import matlab.engine
import matlab

#def Granger (pair):

# Prediction parameters
trialdur = 3000
timewin = 200 #ms
order   = 27

# Time windows to evaluate
times2 = np.linspace(-100, 700, num=21, endpoint=True)#ms
times2save = np.empty((1,times2.size))
times2save[0,:] = times2
#print(times2save.shape)

# Frequency Parameters
min_freq = 2
max_freq = 42
num_freq = 40
frex = np.linspace(min_freq, max_freq, num_freq)
order_points = 15

# Parameters to indices
timewin_points = round(timewin/(1000/250))
#print(timewin_points)
order_points   = round(order/(1000/250))
#print(order_points)

# Subtract mean = Detrend the ERP 
avgelec = np.mean(areg, 0)
#print(areg.shape)
elecs = np.asarray([(y - avgelec) for y in areg])
#elecs = np.transpose(temporal, (1,2,0))
#print('elecs')
#print(elecs.shape)

# Convert requested times to indices
times2saveidx = np.empty(times2save.shape)
for t, tim in enumerate(times2save[0,:]):
    times2saveidx[0,t] = math.ceil((((trialdur/2)+tim)*(750))/trialdur)
#print(times2saveidx)

# Initialize
x2y = np.zeros((1, times2save.shape[1]))
y2x = np.zeros((1, times2save.shape[1]))
bic = np.empty((times2save.shape[1], 15))

tf_granger = np.zeros((2,len(frex),times2save.shape[1]))

for i, timep in enumerate(times2save[0,:]):                 
    
    #print(i)
    #data from all trials in this time window --> trials x channels x time-points
    a = times2saveidx[0,i]-np.floor(timewin_points/2)
    b = times2saveidx[0,i]+np.floor(timewin_points/2)-np.mod(timewin_points+1,2)
    #print(a,b)
    temp = np.squeeze(elecs[:,:,int(a):int(b+1)])
    #print(temp.shape)
    
    # Zscore and detrend
    for tr in range(temp.shape[1]):
        #print(tr)
        temp[tr,0,:] = sp.stats.zscore(sp.signal.detrend(np.squeeze(temp[tr,0,:])));
        temp[tr,1,:] = sp.stats.zscore(sp.signal.detrend(np.squeeze(temp[tr,1,:])));
    # Reshape
    temp = np.reshape(temp, (2, timewin_points*areg.shape[0]))
    #print(temp.shape)
    
    # Auto-Regressive models calculated by armorf.m 
    eng = matlab.engine.start_matlab()
    Ax,Ex = eng.armorf(matlab.double(temp[0,:].tolist()),areg.shape[0],timewin_points-1,order_points, nargout=2)
    Ay,Ey = eng.armorf(matlab.double(temp[1,:].tolist()),areg.shape[0],timewin_points-1,order_points, nargout=2)
    Axy,E = eng.armorf(matlab.double(temp.tolist()),areg.shape[0],timewin_points-1,order_points, nargout=2)
    Ax = np.asarray(Ax)
    Ex = np.asarray(Ex)
    Ay = np.asarray(Ay)
    Ey = np.asarray(Ey)
    Axy = np.asarray(Axy)
    E = np.asarray(E)
    #print(Ex)
    #print(Ey)
    #print(E)

    # G-causality Time-Domain
    y2x[0,i] = math.log(Ex/E[0,0])
    x2y[0,i] = math.log(Ey/E[1,1])
    
    # G-causality Freq-Domain
    eyx = E[1,1] - (E[0,1]**2 / E[0,0])
    exy = E[0,0] - (E[1,0]**2 / E[1,1])
    N = E.shape[0]
    
    for f, freq in enumerate(frex):
        H = np.eye(N)
        #print(H.shape)
        
        for m in range(order_points):
            H = H + Axy[:,(m)*N+1:(m+1)*N] * np.exp(-1j*(m+1)*2*np.pi*freq/250)
                                                  
        Hi = np.linalg.inv(H)
        mult = np.dot(E, Hi.T)
        #print(mult.shape)
        S = np.linalg.solve(H, mult) /250
        #print(S.shape)
        
        # G_caus in the Freq. Domain
        tf_granger[0,f,i] = math.log( np.absolute(S[1,1])/ np.absolute(S[1,1]-(Hi[1,0]*exy*np.conj(Hi[1,0]))) / 250)
        tf_granger[1,f,i] = math.log( np.absolute(S[0,0])/ np.absolute(S[0,0]-(Hi[0,1]*eyx*np.conj(Hi[0,1]))) / 250)
        
print(y2x.shape) 
print(x2y.shape) 
print(tf_granger.shape)
fig1 = plt.figure()
plt.plot(times2save[0,:], y2x.T)
plt.plot(times2save[0,:], x2y.T)
fig2 = plt.figure()
ax1 = fig2.add_subplot(2,1,1)
CS = plt.contourf(times2save[0,1:], frex, np.squeeze(tf_granger[0,:,1:41]), 40)
ax2 = fig2.add_subplot(2,1,2)
CS = plt.contourf(times2save[0,1:], frex, np.squeeze(tf_granger[1,:,1:41]), 40)

plt.show()

In [4]:
import numpy as np
import scipy as sp
import mne
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import fft, ifft, arange, stats, signal
#import matlab.engine
#import matlab

from arpymorf import *

#def Granger (pair):

# Prediction parameters
trialdur = 3000
timewin = 200 #ms
order   = 27

# Time windows to evaluate
times2 = np.linspace(-100, 700, num=21, endpoint=True)#ms
times2save = np.empty((1,times2.size))
times2save[0,:] = times2
#print(times2save.shape)

# Frequency Parameters
min_freq = 2
max_freq = 42
num_freq = 40
frex = np.linspace(min_freq, max_freq, num_freq)
order_points = 15

# Parameters to indices
timewin_points = round(timewin/(1000/250))
#print(timewin_points)
order_points   = round(order/(1000/250))
#print(order_points)

# Subtract mean = Detrend the ERP 
avgelec = np.mean(areg, 0)
#print(areg.shape)
elecs = np.asarray([(y - avgelec) for y in areg])
#elecs = np.transpose(temporal, (1,2,0))
#print('elecs')
#print(elecs.shape)

# Convert requested times to indices
times2saveidx = np.empty(times2save.shape)
for t, tim in enumerate(times2save[0,:]):
    times2saveidx[0,t] = math.ceil((((trialdur/2)+tim)*(750))/trialdur)
#print(times2saveidx)

# Initialize
x2y = np.zeros((1, times2save.shape[1]))
y2x = np.zeros((1, times2save.shape[1]))
bic = np.empty((times2save.shape[1], 15))

tf_granger = np.zeros((2,len(frex),times2save.shape[1]))

for i, timep in enumerate(times2save[0,:]):                 
    
    #print(i)
    #data from all trials in this time window --> trials x channels x time-points
    a = times2saveidx[0,i]-np.floor(timewin_points/2)
    b = times2saveidx[0,i]+np.floor(timewin_points/2)-np.mod(timewin_points+1,2)
    #print(a,b)
    temp = np.squeeze(elecs[:,:,int(a):int(b+1)])
    #print(temp.shape)
    
    # Zscore and detrend
    for tr in range(temp.shape[1]):
        #print(tr)
        temp[tr,0,:] = sp.stats.zscore(sp.signal.detrend(np.squeeze(temp[tr,0,:])));
        temp[tr,1,:] = sp.stats.zscore(sp.signal.detrend(np.squeeze(temp[tr,1,:])));
    # Reshape
    temp = np.reshape(temp, (2, timewin_points*areg.shape[0]))
    zz = temp.shape
    
    # Auto-Regressive models calculated by armorf.m 
    #eng = matlab.engine.start_matlab()
    Ax,Ex = arpymorf(temp[0,:],zz[1],areg.shape[0],timewin_points-1,order_points)
    Ay,Ey = arpymorf(temp[1,:],zz[1],areg.shape[0],timewin_points-1,order_points)
    Axy,E = arpymorf(temp,zz[1],areg.shape[0],timewin_points-1,order_points)
    Ax = np.asarray(Ax)
    Ex = np.asarray(Ex)
    Ay = np.asarray(Ay)
    Ey = np.asarray(Ey)
    Axy = np.asarray(Axy)
    E = np.asarray(E)
    #print(Ex)
    #print(Ey)
    #print(E)

    # G-causality Time-Domain
    y2x[0,i] = math.log(Ex/E[0,0])
    x2y[0,i] = math.log(Ey/E[1,1])
    
    # G-causality Freq-Domain
    eyx = E[1,1] - (E[0,1]**2 / E[0,0])
    exy = E[0,0] - (E[1,0]**2 / E[1,1])
    N = E.shape[0]
    
    for f, freq in enumerate(frex):
        H = np.eye(N)
        #print(H.shape)
        
        for m in range(order_points):
            H = H + Axy[:,(m)*N+1:(m+1)*N] * np.exp(-1j*(m+1)*2*np.pi*freq/250)
                                                  
        Hi = np.linalg.inv(H)
        mult = np.dot(E, Hi.T)
        #print(mult.shape)
        S = np.linalg.solve(H, mult) /250
        #print(S.shape)
        
        # G_caus in the Freq. Domain
        tf_granger[0,f,i] = math.log( np.absolute(S[1,1])/ np.absolute(S[1,1]-(Hi[1,0]*exy*np.conj(Hi[1,0]))) / 250)
        tf_granger[1,f,i] = math.log( np.absolute(S[0,0])/ np.absolute(S[0,0]-(Hi[0,1]*eyx*np.conj(Hi[0,1]))) / 250)
        
print(y2x.shape) 
print(x2y.shape) 
print(tf_granger.shape)
fig1 = plt.figure()
plt.plot(times2save[0,:], y2x.T)
plt.plot(times2save[0,:], x2y.T)
fig2 = plt.figure()
ax1 = fig2.add_subplot(2,1,1)
CS = plt.contourf(times2save[0,1:], frex, np.squeeze(tf_granger[0,:,1:41]), 40)
ax2 = fig2.add_subplot(2,1,2)
CS = plt.contourf(times2save[0,1:], frex, np.squeeze(tf_granger[1,:,1:41]), 40)

plt.show()

LinAlgError: Matrix is not positive definite